In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime,timedelta

In [51]:
order_date = datetime(2020,6,1)
lead_time_days = 90 
days_to_next_order = 30  
current_stock_level = 400
stock_in_transit = 600

In [3]:
sales_data = pd.read_csv('sales_data.csv',parse_dates=['date'])

In [30]:
def forecast_sales(start_date: datetime, sales_data: pd.DataFrame):
    
    mean_daily_sales = sales_data[sales_data['date'] < start_date]['sales'].mean()
    return mean_daily_sales

In [52]:
def calculate_new_order (order_date: datetime, lead_time_days: int, days_to_next_order: int, 
                         sales_data: pd.DataFrame, current_stock_level: int, stock_in_transit: int):
    
    #Order 1
    arrival_date = order_date + timedelta(days=lead_time_days) 
    stock = current_stock_level
    
    #Order 2
    order_date_2 = order_date + timedelta(days=days_to_next_order)
    next_arrival_date = order_date_2 + timedelta(days=lead_time_days)
    sales_forecast = forecast_sales(order_date, sales_data)
    stock -= (sales_forecast * days_to_next_order)
    
    #Arrival of the order 1
    sales_forecast = forecast_sales(order_date_2, sales_data)
    stock += (stock_in_transit - sales_forecast * (arrival_date - order_date_2).days)
    
    #Arrival of the order 2
    stock -= sales_forecast * (next_arrival_date - arrival_date).days
    
    return stock

In [54]:
order = calculate_new_order (order_date=order_date, lead_time_days=lead_time_days, days_to_next_order=days_to_next_order, 
                             sales_data=sales_data, current_stock_level=current_stock_level, 
                             stock_in_transit=stock_in_transit)
print('Next order:', abs(min(0,order)), 'units')

Next order: 186.10568347486088 units


We could include costs and benefits in such a way that at the end of each month the value for both would be indicated together with the final benefit. 
- Costs: €10 per unit purchased + $0.25 per unit in stock each month. In addition, the financial cost of 5% TIN would have to be added.
- Profit: €25 for each unit of glasses sold.
- Total profit: Profit - Costs